# Teil 2: Einführung in Federated Learning

Im letzten Abschnitt haben wir uns mit PointerTensors vertraut gemacht, die die zugrunde liegende Infrastruktur erstellen, die wir zum Schutz der Privatsphäre von Deep Learning benötigen. In diesem Abschnitt erfahren Sie, wie Sie mit diesen grundlegenden Tools unseren ersten Deep-Learning-Algorithmus zum Schutz der Privatsphäre, Federated Learning, implementieren.

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)

### Was ist Federated Learning?

Es ist eine einfache und leistungsstarke Methode, um Deep Learning-Modelle zu trainieren. Wenn Sie an Trainingsdaten denken, ist dies immer das Ergebnis eines Erfassungsprozesses. Menschen (über Geräte) erzeugen Daten, indem sie Ereignisse in der realen Welt aufzeichnen. Normalerweise werden diese Daten an einem einzigen zentralen Ort zusammengefasst, sodass Sie ein Modell für maschinelles Lernen trainieren können. Federated Learning stellt dies auf den Kopf!

Anstatt Trainingsdaten in das Modell (einen zentralen Server) zu bringen, bringen Sie das Modell in die Trainingsdaten (wo immer es sich befindet).

Die Idee ist, dass dies jedem, der die Daten erstellt, ermöglicht, die einzige permanente Kopie zu besitzen, und somit die Kontrolle darüber behält, wer jemals Zugriff darauf hat. Ziemlich cool, oder?

# Abschnitt 2.1 - Ein Beispiel für spielerisches Lernen

Beginnen wir damit, ein Spielzeugmodell zentral zu trainieren. Hier geht es um eine einfache wie Modelle bekommen. Wir brauchen zuerst:

- ein Spielzeugdatensatz
- ein Model
- eine grundlegende Trainingslogik zum Trainieren eines Modells, um die Daten anzupassen.

Hinweis: Wenn Ihnen diese API nicht bekannt ist, gehen Sie zu [fast.ai](http://fast.ai) und nehmen Sie an ihrem Kurs teil, bevor Sie mit diesem Tutorial fortfahren.

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [4]:
train()

tensor(0.6514)
tensor(0.4835)
tensor(0.3618)
tensor(0.2716)
tensor(0.2045)
tensor(0.1543)
tensor(0.1167)
tensor(0.0884)
tensor(0.0671)
tensor(0.0509)
tensor(0.0387)
tensor(0.0295)
tensor(0.0225)
tensor(0.0171)
tensor(0.0130)
tensor(0.0100)
tensor(0.0076)
tensor(0.0058)
tensor(0.0044)
tensor(0.0034)


Und da hast du es! Wir haben ein Grundmodell auf herkömmliche Weise trainiert. Alle unsere Daten werden auf unserem lokalen Computer zusammengefasst und wir können sie verwenden, um Aktualisierungen an unserem Modell vorzunehmen. Federated Learning funktioniert jedoch nicht so. Lassen Sie uns dieses Beispiel so ändern, dass es auf die Art und Weise des Federierten Lernens durchgeführt wird!

Also, was brauchen wir:

- ein paar Arbeiter schaffen
- Hinweise auf Trainingsdaten für jeden Mitarbeiter erhalten
- Aktualisierte Trainingslogik für Verbundlernen

     Neue Trainingsschritte:
     - Modell an korrekten Arbeiter senden
     - auf die dort befindlichen Daten trainieren
     - Holen Sie das Modell zurück und wiederholen Sie es mit dem nächsten Arbeiter

In [5]:
import syft as sy
hook = sy.TorchHook(torch)



In [6]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")



In [7]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [8]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [9]:
train()

tensor(0.2889, requires_grad=True)
tensor(4.6312, requires_grad=True)
tensor(1.6807, requires_grad=True)
tensor(0.8786, requires_grad=True)
tensor(1.0805, requires_grad=True)
tensor(0.4585, requires_grad=True)
tensor(0.6363, requires_grad=True)
tensor(0.2632, requires_grad=True)
tensor(0.3731, requires_grad=True)
tensor(0.1522, requires_grad=True)
tensor(0.2191, requires_grad=True)
tensor(0.0881, requires_grad=True)
tensor(0.1290, requires_grad=True)
tensor(0.0511, requires_grad=True)
tensor(0.0761, requires_grad=True)
tensor(0.0296, requires_grad=True)
tensor(0.0451, requires_grad=True)
tensor(0.0172, requires_grad=True)
tensor(0.0268, requires_grad=True)
tensor(0.0100, requires_grad=True)


## Gut gemacht!

Und voilà! Wir trainieren jetzt ein sehr einfaches Deep Learning-Modell mit Federated Learning! Wir senden das Modell an jeden Mitarbeiter, generieren einen neuen Verlauf und bringen den Verlauf dann zurück zu unserem lokalen Server, wo wir unser globales Modell aktualisieren. Niemals in diesem Prozess sehen oder fordern wir Zugriff auf die zugrunde liegenden Trainingsdaten! Wir bewahren die Privatsphäre von Bob und Alice !!!

## Mängel dieses Beispiels

Obwohl dieses Beispiel eine schöne Einführung in Federated Learning ist, weist es dennoch einige große Mängel auf. Insbesondere wenn wir "model.get ()" aufrufen und das aktualisierte Modell von Bob oder Alice erhalten, können wir tatsächlich viel über die Trainingsdaten von Bob und Alice lernen, indem wir ihre Steigungen betrachten. In einigen Fällen können wir ihre Trainingsdaten perfekt wiederherstellen!

Also, was gibt es zu tun? Nun, die erste Strategie, die Menschen anwenden, besteht darin, den Gradienten über mehrere Personen zu mitteln, bevor er auf den zentralen Server hochgeladen wird. Diese Strategie erfordert jedoch eine differenziertere Verwendung von PointerTensor-Objekten. Im nächsten Abschnitt nehmen wir uns also etwas Zeit, um mehr über erweiterte Zeigerfunktionen zu erfahren, und aktualisieren dann dieses Beispiel für Federated Learning

# Herzliche Glückwünsche!!! - Zeit, der Community beizutreten!

Herzlichen Glückwunsch zum Abschluss dieses Notizbuch-Tutorials! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die Repos in der Hauptrolle zu spielen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Sie können jederzeit zur Seite PySyft GitHub Issues gehen und nach "Projekten" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "gute erste Ausgabe" gekennzeichnet sind.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)



### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

[Open Collective Open Collective Page] (https://opencollective.com/openmined)